In [11]:
import pandas as pd

In [12]:
prot_df = pd.read_csv('../PDB_proteins_90pc_identity_6lu7.csv', engine='python', delimiter=',')

In [13]:
prot_df.columns

Index(['pdb_id', 'organism_scientific_name', 'tax_id', 'organism_synonyms',
       'rank', 'genus', 'superkingdom', 'molecule_name', 'molecule_synonym',
       'molecule_type', 'entity_id', 'modified_residue_flag', 'mutation_type',
       'uniprot_accession', 'uniprot_id', 'gene_name', 'interacting_molecules',
       'interacting_ligands', 'compound_id', 'compound_name',
       'compound_systematic_name', 'entity_weight', 'number_of_polymers',
       'number_of_polymer_residues', 'number_of_polymer_entities',
       'number_of_bound_entities', 'number_of_protein_chains',
       'max_observed_residues'],
      dtype='object')

In [16]:
#Not informative
prot_df[["interacting_ligands",'interacting_molecules',"compound_id","compound_name","compound_systematic_name"]]

,interacting_ligands,interacting_molecules,compound_id,compound_name,compound_systematic_name
0,NaN,NaN,NaN,NaN,NaN
1,"MPD : (4S)-2-METHYL-2\,4-PENTANEDIOL",NaN,MPD,"MPD : (4S)-2-METHYL-2\,4-PENTANEDIOL","MPD : (4S)-2-methylpentane-2\,4-diol"
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,ATO : CHLOROACETONE,5-mer peptide of inhibitor,ATO,ATO : CHLOROACETONE,ATO : 1-chloropropan-2-one
6,I12 : N-[(5-METHYLISOXAZOL-3-YL)CARBONYL]-L-AL...,NaN,I12,I12 : N-[(5-METHYLISOXAZOL-3-YL)CARBONYL]-L-AL...,I12 : N-[(5-methylisoxazol-3-yl)carbonyl]-L-al...
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,"CL : CHLORIDE ION,EDO : 1\,2-ETHANEDIOL",NaN,"EDO,CL","EDO : 1\,2-ETHANEDIOL,CL : CHLORIDE ION","EDO : ethane-1\,2-diol,CL : chloride"
